In [ ]:
from langchain_core.prompts import ChatPromptTemplate
from pprint import pprint
from groq import Groq
from dotenv import load_dotenv
from IPython.display import display_markdown
import os
from IPython.display import Markdown
from langchain_mistralai import ChatMistralAI
from typing import TypedDict
from langchain_groq import ChatGroq

### Some free LLM providers you can set up (be mindful of their limits, use delays if needed)

You can use any of the models you want, **as long you're able to complete the assignment** <br>
A word of caution: For tasks involving tool-calling, LLama is terrible. Small mistral/gemini models work fine most of the times.

1. https://console.groq.com/keys
2. https://ai.google.dev/gemini-api/docs/models#experimental
3. https://docs.mistral.ai/models

In [ ]:
from langchain_mistralai import ChatMistralAI
from langchain_openai import ChatOpenAI
from langchain_groq import ChatGroq

load_dotenv()

## Feel free to switch up the models. E.g. Mistral's limits are imposed model-wise, so you switch between small/medium.

llm = ChatMistralAI(api_key=os.getenv("MISTRAL_API_KEY"), model="mistral-small-latest")

# os.environ["GROQ_API_KEY"]= os.getenv("GROQ_API_KEY")
# client = Groq()
# llm = ChatGroq(api_key = os.getenv("GROQ_API_KEY"),model="llama-3.3-70b-versatile")


# llm = ChatOpenAI(
#     model="gemini-flash-lite-latest",
#     api_key=os.getenv("GEMINI_API_KEY"),
#     base_url="https://generativelanguage.googleapis.com/v1beta/openai/"
# )

## **Section 1: The Reflection Pattern with LangGraph**

In this section, you'll learn to build agentic workflows using `LangGraph`, a powerful library from LangChain for creating complex, stateful, and potentially cyclical agent runtimes. Think of it as building a flowchart for your agents.

We'll start with a very simple "A to B" workflow to understand the core concepts, and then you'll apply that knowledge to build a more advanced reflection agent.

### **Part 1: A Beginner's Guide to LangGraph**

Before we build a looping agent, let's understand the basics with a simple, linear task:
1.  **Agent 1 (Poet):** Writes a short poem about the sky.
2.  **Agent 2 (Translator):** Translates that poem into French.

This will teach you the three core components of any LangGraph workflow:

1.  **The State:** A shared object that holds information and is passed between agents.
2.  **The Nodes:** The "workers" or agents in our graph. Each node is a Python function that performs an action.
3.  **The Edges:** The connections that define the path of the workflow, directing the flow from one node to the next.

#### **Step 1: Define the Graph State**

Our state needs to hold the original poem and the translated version. We use a `TypedDict` for this.

In [ ]:
from typing import TypedDict

class PoemWorkflowState(TypedDict):
    """A state that holds the poem and its translation."""
    poem: str
    translated_poem: str

#### **Step 2: Define the Agent Nodes**

Each agent is a function that takes the current `state` as input and returns a dictionary with the fields it wants to update.

In [ ]:
def poet_node(state: PoemWorkflowState):
    """Generates a poem."""
    print("--- ✒️ POET NODE ---")
    prompt = ChatPromptTemplate.from_template("Write the first four (non-identical) stanza of Iqbal's Jawab-e-Shikwa poem in Urdu")
    chain = prompt | llm
    poem_result = chain.invoke({})
    # Return a dictionary to update the 'poem' field in the state
    return {"poem": poem_result.content}

def translator_node(state: PoemWorkflowState):
    """Translates the poem in the state."""
    print("--- 🌐 TRANSLATOR NODE ---")
    # The 'poem' field was populated by the previous node
    poem_to_translate = state["poem"]
    prompt = ChatPromptTemplate.from_template("Translate the following poem into English:\n\n{poem}")
    chain = prompt | llm
    translation_result = chain.invoke({"poem": poem_to_translate})
    # Return a dictionary to update the 'translated_poem' field
    return {"translated_poem": translation_result.content}

#### **Step 3: Wire up the Graph**

Now we define the flowchart: start at the `poet_node`, then go to the `translator_node`, and then end.

In [ ]:
from langgraph.graph import StateGraph, END

# Create a new graph
workflow = StateGraph(PoemWorkflowState)

# Add the two nodes we defined
workflow.add_node("poet", poet_node)
workflow.add_node("translator", translator_node)

# Set the entry point of the workflow
workflow.set_entry_point("poet")

# Define the connections (edges)
# After the 'poet' node, the workflow should go to the 'translator' node
workflow.add_edge("poet", "translator")
# The 'translator' node is the last step, so we connect it to the special END node
workflow.add_edge("translator", END)

# Compile the graph into a runnable app
app = workflow.compile()

#### **Step 4: Run the Workflow**


In [ ]:
final_state = app.invoke({})

print("\n--- ✅ WORKFLOW COMPLETE ---")
print("\nOriginal Poem:")
print(final_state['poem'])
print("\nTranslated Poem:")
print(final_state['translated_poem'])

Note: all these free models are terrible


### **Part 2: Your Task - Build a Reflection Agent** [30 marks]


Now that you understand the basics of `State`, `Nodes`, and `Edges`, you will build the more complex reflection agent. This agent will have a **cyclical** workflow: **Generate -> Reflect -> (Decide) -> Generate...**

**Goal:** Create a workflow that writes a Python script to scrape Hacker News, and then iteratively refines it based on expert critique.

Follow the `TODO` comments below to implement the full graph.


https://console.groq.com/keys

https://docs.langchain.com/oss/python/langgraph

In [ ]:
#let's save mistral credits for later

os.environ["GROQ_API_KEY"]= "your_groq_api_key_here"
client = Groq()
llm = ChatGroq(model="llama-3.3-70b-versatile")

#### **`TODO 1`: Define the Graph State** [5 marks]


In [ ]:
from typing import List, TypedDict

class GraphState(TypedDict):
    task: str
    code: str
    critiques: List[str]
    revisions: int

#### **`TODO 2 & 3`: Implement the Agent Nodes** [10 marks]


In [ ]:
# --- GENERATION NODE ---
def generation_node(state: GraphState):
    """Generates the code based on the current state."""
    print("--- 💻 GENERATING CODE ---")
    task = state["task"]
    critiques = state.get("critiques", [])
    revisions = state.get("revisions", 0)
    if revisions == 0:
        prompt = ChatPromptTemplate.from_template("Write Python code to accomplish the following task:\n\n{task}")
        chain = prompt | llm
        code_result = chain.invoke({"task": task})
        return {"code": code_result.content, "revisions": revisions + 1}
    else:
        critiques_text = '\n'.join(critiques)
        prompt = ChatPromptTemplate.from_template(
            "Revise the following Python code based on these critiques:\n\n"
            "Code:\n{code}\n\n"
            "Critiques:\n{critiques}"
        )
        chain = prompt | llm
        code_result = chain.invoke({"code": state["code"], "critiques": critiques_text})
        return {"code": code_result.content, "revisions": revisions + 1}

# --- REFLECTION NODE ---
def reflection_node(state: GraphState):
    """Reflects on the code and provides critiques."""
    print("--- 🤔 REFLECTING ON CODE ---")
    code_to_review = state["code"]
    critiques = state.get("critiques", [])
    prompt = ChatPromptTemplate.from_template(
        "You are a Senior Python Developer. Review the following code and provide constructive critiques:\n\n{code}"
    )
    chain = prompt | llm
    review_result = chain.invoke({"code": code_to_review})
    return {"critiques": critiques + [review_result.content]}


#### **`TODO 4`: Implement the Conditional Edge** [5 marks]


In [ ]:
from langgraph.graph import END

def should_continue(state: GraphState):
    """Determines whether to continue the reflection loop."""
    revisions = state["revisions"]
    if revisions >= 2:
        print("Reached maximum revisions. Ending workflow.")
        return "end"
    else:
        print("Revisions remaining. Continuing workflow.")
        return "continue"

#### **`TODO 5`: Wire Up the Graph** [10 marks]


https://docs.langchain.com/oss/python/langgraph/graph-api#conditional-edges

In [ ]:
# Build the graph
workflow = StateGraph(GraphState)

workflow.add_node("generator", generation_node)
workflow.add_node("reflector", reflection_node)
workflow.set_entry_point("generator")
workflow.add_edge("generator", "reflector")
workflow.add_conditional_edges("reflector", should_continue, {"continue": "generator", "end": END})

# Compile the graph into a runnable app
app = workflow.compile()

In [29]:
task = "Create a Python function using requests and BeautifulSoup that scrapes the titles of the top 5 articles from the Hacker News homepage (https://news.ycombinator.com)."
initial_input = {"task": task} # The initial state only needs the task

final_state = app.invoke(initial_input)

print("\n--- ✨ FINAL, REFINED CODE ---")
display(Markdown(final_state['code']))


--- 💻 GENERATING CODE ---
--- 🤔 REFLECTING ON CODE ---
--- 🤔 REFLECTING ON CODE ---
Revisions remaining. Continuing workflow.
--- 💻 GENERATING CODE ---
Revisions remaining. Continuing workflow.
--- 💻 GENERATING CODE ---
--- 🤔 REFLECTING ON CODE ---
--- 🤔 REFLECTING ON CODE ---
Reached maximum revisions. Ending workflow.

--- ✨ FINAL, REFINED CODE ---
Reached maximum revisions. Ending workflow.

--- ✨ FINAL, REFINED CODE ---


The provided code has been revised to address the critiques mentioned. Here is the updated code with explanations and improvements:

```python
import requests
from bs4 import BeautifulSoup
import logging

# Configure logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

# Constant variables
HACKER_NEWS_URL = "https://news.ycombinator.com"
NUM_ARTICLES = 5

def scrape_hacker_news(url: str = HACKER_NEWS_URL, num_articles: int = NUM_ARTICLES) -> list:
    """
    Scrapes the titles of the top articles from the Hacker News homepage.

    Args:
        url (str): The URL of the Hacker News homepage. Defaults to HACKER_NEWS_URL.
        num_articles (int): The number of articles to scrape. Defaults to NUM_ARTICLES.

    Returns:
        list: A list of article titles.
    """
    try:
        # Send a GET request to the Hacker News homepage
        headers = {"User-Agent": "Hacker News Article Scraper"}
        response = requests.get(url, headers=headers)
        response.raise_for_status()  # Raise an exception for HTTP errors

        # Parse the HTML response using BeautifulSoup
        soup = BeautifulSoup(response.text, "lxml")  # Using lxml parser for better performance

        # Find all article titles on the page
        titles = [item.text.strip() for item in soup.find_all("a", class_="storylink")]

        # Return the top articles
        return titles[:num_articles]

    except requests.RequestException as e:
        logger.error(f"Request error: {e}")
        return []
    except Exception as e:
        logger.error(f"An error occurred: {e}")
        return []

def main():
    # Example usage
    article_titles = scrape_hacker_news()
    for i, title in enumerate(article_titles):
        logger.info(f"Article {i+1}: {title}")

if __name__ == "__main__":
    main()
```

### Explanation of Changes

1. **Error Handling**: Added try-except blocks to handle potential errors during the request or parsing process.
2. **User-Agent Header**: Specified a User-Agent header in the request to avoid being blocked by the server.
3. **Constant Variables**: Extracted constant variables for the URL and number of articles to improve readability and maintainability.
4. **Type Hints**: Added type hints for function parameters and return types to enable static type checking and improve code readability.
5. **BeautifulSoup Parser**: Changed the parser to `lxml` for better performance and handling of complex HTML structures.
6. **Logging**: Added logging statements to help with debugging and monitoring the script's execution.

### Improvements

* Improved error handling and logging to make the script more robust and maintainable.
* Used a more efficient parser for better performance.
* Added type hints to improve code readability and enable static type checking.
* Extracted constant variables to improve readability and maintainability.

### Tests and Example Uses

* Run the script to scrape the top 5 article titles from the Hacker News homepage.
* Modify the `HACKER_NEWS_URL` and `NUM_ARTICLES` variables to scrape different numbers of articles or use a different URL.
* Use the `logger` object to log custom messages or errors.

## **Section 2: Tool Calling with LangChain**


An LLM's knowledge is frozen in time and it has no access to the outside world. To build truly powerful applications, we need to give our agents **tools**—functions they can call to interact with APIs, databases, or any other external system.

LangChain provides a seamless way to equip agents with tools and let them decide when to use them.

### **Part 1: A Beginner's Guide to Tool Calling**

Let's start with a very simple example: giving an agent a calculator.

This will teach you the three key components of a LangChain tool-calling agent:
1.  **The Tool:** A Python function decorated with `@tool`.
2.  **The Agent:** The "brain" that decides which tool to use. We'll use `create_tool_calling_agent`.
3.  **The AgentExecutor:** The runtime that actually executes the tool calls and passes the results back to the agent.

#### **Step 1: Define a Tool**

Any Python function can become a tool. The magic is in the `@tool` decorator, which automatically converts the function's signature and docstring into a format the LLM can understand.

> **Important:** A clear, descriptive docstring is crucial. The agent uses the docstring to figure out *what the tool does* and *when to use it*.


In [30]:
from langchain_core.tools import tool

@tool
def multiply(a: int, b: int) -> int:
    """Multiplies two integers together."""
    print("Bro needs to multiply", a, "and", b)
    return a * b

# We create a list of all the tools the agent will have access to.
tools = [multiply]

#### **Step 2: Create a Tool-Calling Agent and Executor**


Now we assemble the agent. We need the LLM, our list of tools, and a special prompt.

The prompt is the agent's instruction manual. We'll use a pre-built template from LangChain which includes a special placeholder: `agent_scratchpad`. This is where the agent will keep track of its internal thoughts and previous tool calls.

In [ ]:
from langchain.agents import create_agent
from langchain_groq import ChatGroq

# Initialize the LLM
llm = ChatGroq(api_key = os.getenv("GROQ_API_KEY"),model="llama-3.3-70b-versatile")

# The agent is created with the model, tools, and a system prompt.
# This single object is now the complete, runnable agent.
agent = create_agent(
    model=llm,
    tools=tools,
    system_prompt="You are a helpful assistant. You must use your tools to answer questions."
)

#### **Step 3: Run the Agent**


Let's ask a question that requires the agent to use its `multiply` tool.


In [32]:
result = agent.invoke({
    "messages": [
        {"role": "user", "content": "What is 8 times 7?"}
    ]
})

# The final answer is in the 'content' of the last message in the output.
final_answer = result["messages"][-1].content
print(final_answer)

Bro needs to multiply 8 and 7
The answer to 8 times 7 is 56.
The answer to 8 times 7 is 56.


### **Part 2: Your Task - Build a Multi-Tool Travel Agent [10 marks]**


You will use the modern `create_agent` function to build a "Travel Agent" that can use **multiple tools** to answer a complex user query.

**Goal:** Create a single agent that can help a user plan a trip by providing information on flights, weather, and local events.

Follow the `TODO` comments below to implement the full agent.

#### **`TODO 1`: Create the Tools [5 marks]**
Define three distinct Python functions. Since we don't have real APIs for this, you will create **mock functions** that return hardcoded string data. Each function must have a clear docstring explaining what it does.

In [ ]:
import json
from langchain_core.tools import tool

@tool
def get_flight_info(origin: str, destination: str, month: str) -> str:
    """Provides fictional flight prices and availability for a trip."""
    return json.dumps({
        "origin": origin,
        "destination": destination,
        "month": month,
        "flights": [
            {"airline": "Pakistan International Airlines", "price": 350, "availability": "Available"},
            {"airline": "FlyJinnah", "price": 400, "availability": "Limited Seats"},
            {"airline": "Air Sial", "price": 300, "availability": "Sold Out"}
        ]
    })

@tool
def get_weather_forecast(city: str, month: str) -> str:
    """Provides a fictional weather forecast for a specific city and month."""
    return json.dumps({
        "city": city,
        "month": month,
        "forecast": {
            "average_high": "30°C",
            "average_low": "20°C",
            "precipitation": "50mm"
        }
    })

@tool
def search_city_events(city: str, month: str) -> str:
    """Provides a list of major fictional events for a specific city and month."""
    return json.dumps({
        "city": city,
        "month": month,
        "events": [
            {"name": "Hasan Raheem Concert", "date": f"{month}-15", "description": "A grand music festival featuring Hasan Raheem."},
            {"name": "Pakwheels Car Mela", "date": f"{month}-22", "description": "A car mela where you can buy and sell vehicles."},
            {"name": "Topics in LLMs Final Presentation", "date": f"{month}-28", "description": "The final..."}
        ]
    })

travel_tools = [get_flight_info, get_weather_forecast, search_city_events]

#### **`TODO 2`: Create and Run the Agent 🚀 [5 marks]**

In [ ]:
from langchain.agents import create_agent
from langchain_groq import ChatGroq

# 1. Initialize the LLM 
llm = ChatGroq(api_key = os.getenv("GROQ_API_KEY"),model="llama-3.3-70b-versatile")

# 2. Define a clear and concise system prompt for your travel agent.
system_prompt = "You are a travel planning assistant. Use the available tools to help users plan their trips by providing information on flights, weather, and local events."

# 3. Create the agent using create_agent, passing the llm, tools list, and system prompt.
agent = create_agent(
    model=llm,
    tools=travel_tools,
    system_prompt=system_prompt
)

user_message_content = "Help me plan a trip to Tel Aviv from Tehran for this June. I need to know about flights, weather, and any major events."

# 5. Invoke the agent with the correct message format and print the final answer.
result = agent.invoke({
    "messages": [
        {"role": "user", "content": user_message_content}
    ]
})
final_answer = result["messages"][-1].content
print(final_answer)

For your trip to Tel Aviv from Tehran in June, there are several flights available with prices ranging from $300 to $400. The weather in Tel Aviv during June is expected to be warm with average highs of 30°C and average lows of 20°C, with a precipitation of 50mm. As for major events, there are a few happening in Tel Aviv during June, including the Hasan Raheem Concert on June 15th, the Pakwheels Car Mela on June 22nd, and the Topics in LLMs Final Presentation on June 28th.


## **Section 3: Advanced Multi-Agent Collaboration with LangGraph**


You have now mastered the core patterns of agentic design: stateful workflows with `LangGraph`, tool use with `create_agent`, and multi-agent collaboration. This final project will challenge you to combine all these skills to build a sophisticated, practical research crew.

Your goal is to create a multi-agent system that can verify a claim by consulting multiple sources, cross-referencing their findings, and looping its research until it reaches a confident conclusion.

### **Using the Tavily Search Tool**
For this task, we will use the **Tavily Search API** for live web searches. It is a powerful tool designed specifically for LLM agents.


https://www.tavily.com/

In [43]:
import os
from dotenv import load_dotenv

# Load environment variables from .env file
load_dotenv()

# It is recommended to use a secrets manager for your keys.
os.environ["TAVILY_API_KEY"] = os.getenv("TAVILY_API_KEY")


Also, you're free to use any model here on. But to warn you, several models (e.g. LLama we used earlier) are absolutely terrible at calling tools appropriately. Mistral should work fine, most of the times.

In [44]:
from langchain_tavily import TavilySearch
from langchain.agents import create_agent
from langchain_groq import ChatGroq

# You can configure the tool's parameters, like max_results, upon instantiation
search_tool = TavilySearch(max_results=3)


# You can then pass this tool in a list to your agent like this and it will automatically perform a search if needed
llm = ChatMistralAI(api_key=os.getenv("MISTRAL_API_KEY"), model="mistral-small-latest")
my_agent = create_agent(
    model=llm,
    tools=[search_tool], # Pass the tool here
    system_prompt="You are a web researcher."
)

### **Your Task: Build the Fact-Checking Crew [30 marks]**

#### **High-Level Architecture**

Your crew will now include a skeptical agent to ensure a balanced perspective.
1.  **Start with a claim.**
2.  **Initial Research:** A news-focused agent will search for evidence supporting the claim.
3.  **Adversarial Research:** A "Devil's Advocate" agent will actively search for evidence that *contradicts* the claim.
4.  **Verification:** A lead verifier will analyze both the supporting and contradictory evidence, produce a consolidated analysis, and decide on a verdict: `CONFIRMED`, `CONTRADICTED`, or `NEEDS_MORE_INFO`.
5.  **Conditional Loop:** If the verdict is `NEEDS_MORE_INFO`, the graph loops back for another round of research.
6.  **Final Report:** A writer agent takes the final, verified analysis and produces a polished report.

#### **`TODO 1`: Define the Graph State [5 marks]**
The state needs to track the claim, the findings from the pro and con agents, the verifier's analysis, and the final report.

In [ ]:
from typing import TypedDict, List

class FactCheckCrewState(TypedDict):
    claim: str
    revision_number: int
    supporting_evidence: str
    counter_evidence: str
    verified_analysis: str
    final_verdict: str
    final_report: str

#### **`TODO 2`: Define the Agent Nodes [15 marks]**
You will now create four distinct agent nodes.

In [ ]:
from langchain.agents import create_agent
from langchain_core.prompts import ChatPromptTemplate
from langchain_groq import ChatGroq
from langchain_tavily import TavilySearch
import json

#Use this search tool to enable live searches
search_tool = TavilySearch(max_results=4)

load_dotenv()

llm = ChatMistralAI(api_key=os.getenv("MISTRAL_API_KEY"), model="mistral-small-latest")

# --- SUPPORTING EVIDENCE AGENT ---
def supporting_evidence_node(state: FactCheckCrewState):
    """Agent 1: Uses the search tool to find recent news and articles that SUPPORT the claim."""
    print("--- 🔎 SUPPORTING EVIDENCE NODE ---")
    support_agent = create_agent(
        model=ChatMistralAI(api_key=os.getenv("MISTRAL_API_KEY"), model="mistral-small-latest"),
        tools=[search_tool],
        system_prompt="You are a research assistant. Find credible news, studies, and expert opinions that support the claim."
    )
    result = support_agent.invoke({
        "messages": [
            {"role": "user", "content": state["claim"]}
        ]
    })
    supporting_evidence = result["messages"][-1].content
    return {"supporting_evidence": supporting_evidence}

# --- DEVIL'S ADVOCATE AGENT ---
def devils_advocate_node(state: FactCheckCrewState):
    """Agent 2: Uses the search tool to find evidence that CONTRADICTS the claim."""
    print("--- 😈 DEVIL'S ADVOCATE NODE ---")
    devil_agent = create_agent(
        model=ChatMistralAI(api_key=os.getenv("MISTRAL_API_KEY"), model="mistral-small-latest"),
        tools=[search_tool],
        system_prompt="You are a Devil's Advocate. Find counter-arguments, dissenting opinions, and evidence that debunks or challenges the claim."
    )
    result = devil_agent.invoke({
        "messages": [
            {"role": "user", "content": state["claim"]}
        ]
    })
    counter_evidence = result["messages"][-1].content
    return {"counter_evidence": counter_evidence}

# --- LEAD VERIFIER AGENT ---
def lead_verifier_node(state: FactCheckCrewState):
    """Agent 3: Synthesizes both sides and makes a verdict. Does not use tools."""
    print("--- ⚖️ LEAD VERIFIER NODE ---")
    prompt = ChatPromptTemplate.from_template(
        "You are a neutral judge. Weigh the following evidence:\n\n"
        "Supporting Evidence:\n{supporting_evidence}\n\n"
        "Counter Evidence:\n{counter_evidence}\n\n"
        "You MUST respond with ONLY a valid JSON object (no other text before or after). "
        "The JSON must have exactly two keys:\n"
        '- "analysis": A summary of the verified findings.\n'
        '- "verdict": One of exactly these three values: "CONFIRMED", "CONTRADICTED", or "NEEDS_MORE_INFO".\n\n'
        "Example format:\n"
        '{{"analysis": "Your analysis here", "verdict": "CONFIRMED"}}'
        'This format MUST be followed strictly.'
    )
    chain = prompt | ChatMistralAI(api_key=os.getenv("MISTRAL_API_KEY"), model="mistral-small-latest")
    verification_result = chain.invoke({
        "supporting_evidence": state["supporting_evidence"],
        "counter_evidence": state["counter_evidence"]
    })
    # Clean up the response - remove markdown code blocks if present
    content = verification_result.content.strip()
    if content.startswith("```json"):
        content = content[7:]  # Remove ```json
    if content.startswith("```"):
        content = content[3:]  # Remove ```
    if content.endswith("```"):
        content = content[:-3]  # Remove trailing ```
    content = content.strip()
    
    output_json = json.loads(content)
    return {
        "verified_analysis": output_json["analysis"],
        "final_verdict": output_json["verdict"],
        "revision_number": state["revision_number"] + 1
    }

# --- REPORT WRITER AGENT ---
def report_writer_node(state: FactCheckCrewState):
    """Agent 4: Writes the final, polished report."""
    print("--- 🖋️ REPORT WRITER NODE ---")
    prompt = ChatPromptTemplate.from_template(
        "You are a report writer. Based on the following analysis and verdict, write a clear, neutral, and well-structured report for a general audience:\n\n"
        "Analysis:\n{verified_analysis}\n\n"
        "Verdict:\n{final_verdict}"
    )
    chain = prompt | ChatMistralAI(api_key=os.getenv("MISTRAL_API_KEY"), model="mistral-small-latest")
    report_result = chain.invoke({
        "verified_analysis": state["verified_analysis"],
        "final_verdict": state["final_verdict"]
    })
    return {"final_report": report_result.content}


#### **`TODO 3`: Implement the Conditional Edge [5 marks]**
This function will read the `final_verdict` and decide the next step for the graph.

In [ ]:
def should_continue_verification(state: FactCheckCrewState):
    """Determines whether to loop back for more research or finish."""
    print("--- 🔁 DECISION NODE ---")
    final_verdict = state["final_verdict"]
    revision_number = state["revision_number"]
    if final_verdict == "NEEDS_MORE_INFO" and revision_number < 2:
        print("More research needed. Continuing verification loop.")
        return "continue_research"
    else:
        print("Sufficient information gathered. Finishing report.")
        return "finish_report"

#### **`TODO 4`: Wire Up the Graph [5 marks]**


In [ ]:
from langgraph.graph import StateGraph, END

workflow = StateGraph(FactCheckCrewState)
workflow.add_node("supporting_evidence", supporting_evidence_node)
workflow.add_node("devils_advocate", devils_advocate_node)
workflow.add_node("lead_verifier", lead_verifier_node)
workflow.add_node("report_writer", report_writer_node)
workflow.set_entry_point("supporting_evidence")
workflow.add_edge("supporting_evidence", "devils_advocate")
workflow.add_edge("devils_advocate", "lead_verifier")
workflow.add_conditional_edges("lead_verifier", should_continue_verification, {"continue_research": "supporting_evidence", "finish_report": "report_writer"})
workflow.add_edge("report_writer", END)

app = workflow.compile()

#### **Run Your Completed Crew**


In [82]:
claim = "Pakistan gained independence in 1947."
initial_input = {"claim": claim, "revision_number": 0}

final_state = app.invoke(initial_input)

print("\n\n--- ✅ FINAL REPORT ---")
print(final_state['final_report'])

--- 🔎 SUPPORTING EVIDENCE NODE ---
--- 😈 DEVIL'S ADVOCATE NODE ---
--- ⚖️ LEAD VERIFIER NODE ---
--- 🔁 DECISION NODE ---
Sufficient information gathered. Finishing report.
--- 🖋️ REPORT WRITER NODE ---


--- ✅ FINAL REPORT ---
**Report: Confirmation of Pakistan's Independence**

**Introduction**

This report aims to provide a clear and concise overview of the historical fact of Pakistan's independence. Following a thorough analysis of available evidence and historical records, we are able to confirm the country's independence in 1947.

**Key Findings**

The evidence confirms that Pakistan gained independence on August 14, 1947. This date is widely recognized and supported by historical events, including the swearing-in of Muhammad Ali Jinnah as the first governor general of Pakistan. Additionally, the release of commemorative postage stamps on this occasion further substantiates the fact of Pakistan's independence.

**Historical Context**

While there may be ongoing discussions and deb

## **Section 4: A Comparative Study of Fact-Checking Agents** [20 marks]

How much better is a complex agent than a simple one? You will answer that question by empirically evaluating three different fact-checking methods against a real-world dataset of claims.

#### **The Goal**

You will take a dataset of fact-checked claims and run each claim through three different verifiers:
1.  **Method 1: The Zero-Shot LLM:** A baseline agent with no tools, relying solely on its internal knowledge.
2.  **Method 2: The Simple Search Agent:** A single agent equipped with a web search tool (a basic RAG approach).
3.  **Method 3: The Advanced Research Crew:** The multi-agent, adversarial fact-checking crew you just built.

Finally, you will compare the accuracy of each method to determine the value of agentic complexity.

#### **Setup: Loading the Dataset**


In [5]:
import pandas as pd
from tqdm import tqdm

try:
    df = pd.read_csv('claims.csv')
    claims_sample = df[:10].copy()
except FileNotFoundError:
    print("Error: 'claims.csv' not found.")
    claims_sample = pd.DataFrame()

def normalize_verdict(verdict):
    if verdict:
        return "true"
    else:
        return "false"
    

if not claims_sample.empty:
    claims_sample['ground_truth'] = claims_sample['text review'].apply(normalize_verdict)

Note: You might want to enforce structured outputs to ensure your final answer is a True/False: <br> 
https://docs.langchain.com/oss/python/langchain/structured-output <br>
https://forum.langchain.com/t/make-a-llm-with-structured-output-call-a-tool/622

### **Task 1: Baseline Fact-Checker (Zero-Shot LLM) [5 marks]**

This agent has no access to the outside world. It will make its judgment based only on the information it was trained on.

**Your Task:** Implement the `verify_claim_zero_shot` function. This function should use a simple LLM chain to classify a claim as "True" or "False".


In [ ]:
# All our necessary imports from previous sections
from langchain.agents import create_agent
from langchain_core.prompts import ChatPromptTemplate
from langchain_groq import ChatGroq
from langchain_tavily import TavilySearch
from langchain_openai import ChatOpenAI
from dotenv import load_dotenv
from langchain_mistralai import ChatMistralAI
import os

load_dotenv()


llm = ChatMistralAI(api_key=os.getenv("MISTRAL_API_KEY"), model="mistral-small-latest")
search_tool = TavilySearch(max_results=4)

def verify_claim_zero_shot(claim: str) -> str:
    """Verifies a claim using only the LLM's internal knowledge."""
    prompt = ChatPromptTemplate.from_template(
        "You are a fact-checker. Verify the following claim:\n\n{claim}\n\n"
        "Respond with ONLY the word 'True' if the claim is correct, or 'False' if it is incorrect."
    )
    chain = prompt | llm
    result = chain.invoke({"claim": claim})
    output = result.content.strip().lower()
    if "true" in output:
        return "True"
    else:
        return "False"

# --- Run the evaluation ---
print("Running Zero-Shot Verifier...")
results_zero_shot = []
for claim in tqdm(claims_sample['claim'], desc="Zero-Shot Verification"):
    verdict = verify_claim_zero_shot(claim)
    results_zero_shot.append(verdict)

claims_sample['zero_shot_verdict'] = results_zero_shot

Running Zero-Shot Verifier...


Zero-Shot Verification: 100%|██████████| 10/10 [00:03<00:00,  3.22it/s]


### **Task 2: Simple RAG Fact-Checker (Agent with Search Tool) [5 marks]**
This agent represents a standard RAG (Retrieval-Augmented Generation) approach. It can search the web for information before making a decision.


**Your Task:** Implement the `verify_claim_with_search` function. This function will create a simple agent equipped with the `TavilySearch` tool.


In [ ]:
import time

load_dotenv()


llm = ChatMistralAI(api_key=os.getenv("MISTRAL_API_KEY"), model="mistral-small-latest")

def verify_claim_with_search(claim: str) -> str:
    """Verifies a claim using a single agent with a web search tool."""
    agent = create_agent(
        model=llm,
        tools=[search_tool],
        system_prompt="You are a fact-checker with access to a web search tool. Use the tool to find information and make a final judgment on the claim. You MUST respond with ONLY the word 'True' or 'False'."
    )
    result = agent.invoke({
        "messages": [
            {"role": "user", "content": claim}
        ]
    })
    final_output = result["messages"][-1].content.strip().lower()
    if "true" in final_output:
        return "True"
    else:
        return "False"

# --- Run the evaluation ---
print("\nRunning Simple Search Verifier...")
results_with_search = []
for claim in tqdm(claims_sample['claim'], desc="Search Verification"):
    verdict = verify_claim_with_search(claim)
    results_with_search.append(verdict)
    time.sleep(20)  # Add a delay to avoid hitting rate limits


claims_sample['simple_search_verdict'] = results_with_search


Running Simple Search Verifier...


Search Verification: 100%|██████████| 10/10 [04:29<00:00, 27.00s/it]


### **Task 3: Advanced Fact-Checker (Multi-Agent Crew) [5 marks]**

**You may re-use the multi-agent crew you built in the previous section.**

**Your Task:** Implement the `verify_claim_with_crew` function. This function will invoke your crew and, most importantly, translate its complex output (`CONFIRMED`, `CONTRADICTED`, `NEEDS_MORE_INFO`) into the simple "True"/"False" format required for evaluation.

Note: Restrict the number of revisions to 2. If the crew is not able to arrive at a definitive answer, output 'uncertain'.  I'll leave it upto you to think and decide if such a behavior should be penalized. You may compute your accuracy accordingly.

In [ ]:
load_dotenv()


llm = ChatMistralAI(api_key=os.getenv("MISTRAL_API_KEY"), model="mistral-small-latest")

workflow = StateGraph(FactCheckCrewState)
workflow.add_node("supporting_evidence", supporting_evidence_node)
workflow.add_node("devils_advocate", devils_advocate_node)
workflow.add_node("lead_verifier", lead_verifier_node)
workflow.add_node("report_writer", report_writer_node)
workflow.set_entry_point("supporting_evidence")
workflow.add_edge("supporting_evidence", "devils_advocate")
workflow.add_edge("devils_advocate", "lead_verifier")
workflow.add_conditional_edges("lead_verifier", should_continue_verification, {"continue_research": "supporting_evidence", "finish_report": "report_writer"})
workflow.add_edge("report_writer", END)

app = workflow.compile()

def verify_claim_with_crew(claim: str) -> str:
    """Verifies a claim using the full multi-agent research crew."""
    initial_state = {"claim": claim, "revision_number": 0}
    final_state = app.invoke(initial_state)
    final_verdict = final_state["final_verdict"]
    if final_verdict == "CONFIRMED":
        return "True"
    elif final_verdict == "CONTRADICTED":
        return "False"
    else:
        return "Other"

# --- Run the evaluation ---
print("\nRunning Multi-Agent Crew Verifier...")
results_with_crew = []
# Ensure your crew's LangGraph `app` is defined and compiled in a cell above this one!
for claim in tqdm(claims_sample['claim'], desc="Crew Verification"):
    verdict = verify_claim_with_crew(claim)
    results_with_crew.append(verdict)
    time.sleep(60)

claims_sample['crew_verdict'] = results_with_crew


Running Multi-Agent Crew Verifier...


Crew Verification:   0%|          | 0/10 [00:00<?, ?it/s]

--- 🔎 SUPPORTING EVIDENCE NODE ---
--- 😈 DEVIL'S ADVOCATE NODE ---
--- 😈 DEVIL'S ADVOCATE NODE ---
--- ⚖️ LEAD VERIFIER NODE ---
--- ⚖️ LEAD VERIFIER NODE ---
--- 🔁 DECISION NODE ---
More research needed. Continuing verification loop.
--- 🔎 SUPPORTING EVIDENCE NODE ---
--- 🔁 DECISION NODE ---
More research needed. Continuing verification loop.
--- 🔎 SUPPORTING EVIDENCE NODE ---
--- 😈 DEVIL'S ADVOCATE NODE ---
--- 😈 DEVIL'S ADVOCATE NODE ---
--- ⚖️ LEAD VERIFIER NODE ---
--- ⚖️ LEAD VERIFIER NODE ---
--- 🔁 DECISION NODE ---
Sufficient information gathered. Finishing report.
--- 🖋️ REPORT WRITER NODE ---
--- 🔁 DECISION NODE ---
Sufficient information gathered. Finishing report.
--- 🖋️ REPORT WRITER NODE ---


Crew Verification:  10%|█         | 1/10 [01:33<13:58, 93.19s/it]

--- 🔎 SUPPORTING EVIDENCE NODE ---
--- 😈 DEVIL'S ADVOCATE NODE ---
--- 😈 DEVIL'S ADVOCATE NODE ---
--- ⚖️ LEAD VERIFIER NODE ---
--- ⚖️ LEAD VERIFIER NODE ---
--- 🔁 DECISION NODE ---
More research needed. Continuing verification loop.
--- 🔎 SUPPORTING EVIDENCE NODE ---
--- 🔁 DECISION NODE ---
More research needed. Continuing verification loop.
--- 🔎 SUPPORTING EVIDENCE NODE ---
--- 😈 DEVIL'S ADVOCATE NODE ---
--- 😈 DEVIL'S ADVOCATE NODE ---


...

Crew Verification:  80%|████████  | 8/10 [13:10<02:55, 87.76s/it]

--- 🔎 SUPPORTING EVIDENCE NODE ---
--- 😈 DEVIL'S ADVOCATE NODE ---
--- 😈 DEVIL'S ADVOCATE NODE ---
--- ⚖️ LEAD VERIFIER NODE ---
--- ⚖️ LEAD VERIFIER NODE ---
--- 🔁 DECISION NODE ---
Sufficient information gathered. Finishing report.
--- 🖋️ REPORT WRITER NODE ---
--- 🔁 DECISION NODE ---
Sufficient information gathered. Finishing report.
--- 🖋️ REPORT WRITER NODE ---


Crew Verification:  90%|█████████ | 9/10 [14:25<01:24, 84.00s/it]

--- 🔎 SUPPORTING EVIDENCE NODE ---
--- 😈 DEVIL'S ADVOCATE NODE ---
--- 😈 DEVIL'S ADVOCATE NODE ---
--- ⚖️ LEAD VERIFIER NODE ---
--- ⚖️ LEAD VERIFIER NODE ---
--- 🔁 DECISION NODE ---
Sufficient information gathered. Finishing report.
--- 🖋️ REPORT WRITER NODE ---
--- 🔁 DECISION NODE ---
Sufficient information gathered. Finishing report.
--- 🖋️ REPORT WRITER NODE ---


Crew Verification: 100%|██████████| 10/10 [15:41<00:00, 94.16s/it]


### **Final Evaluation & Analysis [5 marks]**

Now for the moment of truth. We will calculate the accuracy of each method by comparing its verdicts to the ground truth from the dataset.

**Your Task:** Run the evaluation code and then, in the final markdown cell, write a brief analysis of the results.

In [77]:
# make all the final verdict columns all lowercase for consistency
claims_sample['zero_shot_verdict'] = claims_sample['zero_shot_verdict'].str.lower()
claims_sample['simple_search_verdict'] = claims_sample['simple_search_verdict'].str.lower()
claims_sample['crew_verdict'] = claims_sample['crew_verdict'].str.lower()

# --- Calculate Accuracy ---
ground_truth = claims_sample['ground_truth'].values

accuracy_zero_shot = (claims_sample['zero_shot_verdict'].values == ground_truth).mean()
accuracy_simple_search = (claims_sample['simple_search_verdict'].values == ground_truth).mean()
accuracy_crew = (claims_sample['crew_verdict'].values == ground_truth).mean()

print("--- Final Results ---")
print(f"Zero-Shot LLM Accuracy: {accuracy_zero_shot:.2%}")
print(f"Simple Search Agent Accuracy: {accuracy_simple_search:.2%}")
print(f"Multi-Agent Crew Accuracy: {accuracy_crew:.2%}")

print("\n--- Detailed Comparison ---")
display(claims_sample[['claim', 'ground_truth', 'zero_shot_verdict', 'simple_search_verdict', 'crew_verdict']])

--- Final Results ---
Zero-Shot LLM Accuracy: 80.00%
Simple Search Agent Accuracy: 90.00%
Multi-Agent Crew Accuracy: 40.00%

--- Detailed Comparison ---


,claim,ground_truth,zero_shot_verdict,simple_search_verdict,crew_verdict
0,Do researchers predict 'Medicare for All' woul...,true,true,true,true
1,Fotoğrafın Binali Yıldırım'a verilen tablodaki...,false,false,false,other
2,"Abimola, 36 anni, congolese pregiudicato, vive...",false,false,false,other
3,Most affected countries by COVID-19 are along ...,false,false,false,false
4,Intervenção artística em Arroios custou 37 mil...,true,false,false,other
5,Donald Trump Pakistan’da mı doğdu?,false,false,false,true
6,An image shared on Instagram purportedly shows...,false,false,false,false
7,സർവർക്കർ മാപ്പ് പറഞ്ഞെന്ന് 1947 ൽ ജനഭൂമി വാർത്ത,false,false,false,other
8,Photograph shows an eagle perched atop a grave...,true,false,true,true
9,Infiltrado do MST é gravado quebrando vidraças...,false,false,false,true


#### **Analysis of Results**

**1. Which method performed the best? Why?**

The Simple Search Agent performed the best with 90% accuracy, followed by Zero-shot LLM at 80%, while the Multi-Agent Crew underperformed at 40%. 

The Simple Search Agent's success stems from its balanced and streamlined approach: it combines the LLM's reasoning with real-time web access for claim verification. Its singular focus (search $\rightarrow$ analyze $\rightarrow$ decide) is efficient and avoids the errors introduced by complex multi-agent handoffs.

**2. Do you think these results were expected?**

The results are unexpected, particularly the Multi-Agent Crew's low performance. Theoretically, its adversarial reasoning (supporting vs. contradicting evidence) should improve performance.

However, factors contributing to the underperformance likely include:

- **Increased complexity:** More agents and handoffs introduce more failure points and opportunities for miscommunication.

- **Verdict translation challenges:** The crew's nuanced verdicts (CONFIRMED/CONTRADICTED/NEEDS_MORE_INFO) don't map cleanly to a binary True/False, especially when uncertain.

- **Conflicting evidence:** The adversarial design may find contradictory information even for straightforward claims, leading to false "NEEDS_MORE_INFO" verdicts.

The Simple Search Agent's superior performance suggests that for binary fact-checking, a focused single-agent approach with tool access is often more effective than complex multi-agent architectures.

**3. If the multi-agent crew was not able to output a final answer - what's better from a social welfare perspective: an output you are not confident about or refraining from giving an output if you are not confident?**

From a social welfare perspective, refraining from giving an output when confidence is low is generally the more ethical choice. It prevents misinformation and builds long-term trust. A confident incorrect verdict is more harmful than admitting uncertainty.